## Энергетический оракул
Ноутбук команды #12

Работа выполнена на основе модели LightGBM


### 1. Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

from xgboost import XGBRegressor
from xgboost.callback import TrainingCallback

from catboost import CatBoostRegressor

import re

from tqdm import tqdm
import time

random_state = 12345
NUM_ITERATIONS = 5000

#### 1.1 Функции для расшифровки прогноза погоды в колонке 'weather_pred'

In [2]:
# Расшифровка прогноза в колонке 'weather_pred'

# функция формирует колонки 'cloudy', 'rainy', 'windy', 'clear', 'rain_probability', 'has_rain_probability'
# в колонках число, которое 0 при отсутсвии упоминания явления в weather_pred или степень упоминания
# функция дает в колонках номер первого списка, элемент которого есть в строке плюс 1
# списки cloudy_list, rainy_list, windy_list, clear_list можно модифицировать
# соответственно, можно экспериментировать с расположением значений в списках
# например, сейчас 'дождь', 'снег', 'д+сн' - первая степень  дождя, а 'гроз', 'ливень' - вторая
# а можно сделать снег второй, а грозу с ливнем убрать в третью
# также сделал отдельный список для "ясности", чтобы выделить 'ясно' и 'солнечно'

def in_what_list(weather, big_list):
    for list_number, small_list in enumerate(big_list):
        if any(word in weather for word in small_list):
            return list_number+1
    return 0

def weather_split2(row):
    weather = row['weather_pred']
    cloudy_list = [['проясн', 'пер.об.', 'п/об'], ['пасм', 'обл']]
    rainy_list = [['дождь', 'снег', 'д+сн'], ['гроз', 'ливень']]
    windy_list = [['вет'],['штор']]
    clear_list = [['проясн'], ['ясно'], ['солнеч']]
    numbers = re.findall(r'\d+', weather)
    cloudy = in_what_list(weather, cloudy_list)
    rainy = in_what_list(weather, rainy_list)
    windy = in_what_list(weather, windy_list)
    clear = in_what_list(weather, clear_list)
    rain_probability = 0 if len(numbers)==0 else int(numbers[0])
    has_rain_probability = int(len(numbers)==0)
    return cloudy, rainy, windy, clear, rain_probability, has_rain_probability

def fill_weather_columns(df):
    df['weather_pred'] = df['weather_pred'].fillna('')
    df['cloudy'], df['rainy'], df['windy'], df['clear'], df['rain_probability'], df['has_rain_probability'] = \
                zip(*df.apply(weather_split2, axis=1))
    return df

#### 1.2 Функции для загрузки данных о ВВП 
данные загружаются из файла 'data/VVP.csv'

Некоторые научные работы указывают на прямую связь величины потребления электричества и показателя ВВП, который отражает ситуацию в экономике. Данные по экономике публикуются различными министерствами с разной периодичностью. Для использования в работе были взяты фактические данные по ВВП с сайта investing, который агрегирует публикации Минэкономразвития. Данные за месяц побликуются с месячной задержкой, поэтому модель использует для прогнозирования данные за прошлые месяцы, которые известны.   
  
Ссылка на данные: https://ru.investing.com/economic-calendar/russian-monthly-gdp-407


In [3]:
# Функция добавляет данные о ВВП из файла 'data/VVP.csv' в датасет

def add_vvp2(data, file_source = 'data/VVP.csv'):
    """
    сырой датафрем подаем на вход
    """
    # обработаем файл с динамикой ВВП
    vvp = pd.read_csv(file_source)
    # преобразуем дату файла-источника в формат datetime64 и дропнем один столбик
    vvp['date'] = pd.to_datetime(vvp['date'], format ='%Y-%m-%d %H:%M:%S')
    vvp.drop('for_month',axis=1,inplace=True) 
    
    # обработаем основной фрейм - создадим столбец для соединения, который потом удалим
    data['date_temp'] = pd.to_datetime(data['date'], format = '%Y-%m-%d' )
    data['date_temp'] = data['date_temp'] + pd.to_timedelta(data['time'] , 'H')
    
    # соединяем основной фрейм и ВВП по дате объявления показтеля ВВП
    for idx in reversed(vvp.index):
        data.loc[data['date_temp']>=vvp.date[idx],'VVP'] = vvp.VVP_perc[idx]
        
    data.drop('date_temp',axis=1,inplace=True)   

    return data

#### 1.3 Функции для загрузки архива данных о фактической погоде
данные загружаются из файла 'data/preprocessing_loaded_table.csv'

Изначально данные для формирования таблицы "preprocessing_loaded_table" были взяты из с сайта [https://rp5.ru](https://rp5.ru/Архив_погоды_в_Храброво,_им._императрицы_Елизаветы_Петровны_(аэропорт),_METAR), где хранятся архивы погоды в аэрапорту Калининграда, за период с 31.12.2018 по 30.09.2023

Описание данных в таблице:
- Местное время в Храброво / им. императрицы Елизаветы Петровны (аэропорт) - Дата / Местное время
- T -  Темпиратура воздуха
- Po - Давление на уровне станции
- P - Давление приведённое к уровню моря
- U - Относительная влажность
- DD - Направление ветра
- Ff - Скорость ветра
- ff10 - Максимальное значение порыва ветра
- WW - Особое явление текущей погоды (осадки)
- W'W' - Явление недавней погоды, имеющее оперативное значение
- с - Общая облачность
- VV - Горизонтальная дальность видимости
- Td - Темпиратура точки росы

Данные, которые были взяты из данной таблицы и загружаются из 'data/preprocessing_loaded_table.csv':
- P - не подверглось изменению
- U - не подверглось изменению
- Td - не подверглась изменению

 WW - разделили на 4 категории:
- Нет осадков (где были пропуски)
- слабый дождь
- сильный дождь
- снег

DD - создали 4 столбца, соответствующих сторонам горизонта, которые принимали значения 0; 0.5 и 1 в зависимости от силы ветра в конкретном направлении
- N - north
- S - south
- W - west
- E - east

В дальнейшем эти данные использовались с лагом в сутки: в поля на завтрашний день записывались данные сегодняшнего.

In [4]:
# Функции для работы с данными о фактической погоде из 'data/preprocessing_loaded_table.csv'

# Кодировка информации об осадках из колонки WW
def true_weather_WW_replace(ww):
    if ww=='нет осадков':
        return 0
    elif ww=='слабый дождь':
        return 1
    elif (ww=='сильный дождь') or (ww=='снег'):
        return 2
    else:
        return 3

# Вычисление Timestamp из даты и времени
def row_plus_hours_to_index(row):
    return row['date'] + pd.to_timedelta(row['time'] , 'H')

# Функция для сдвига на сутки (в скачанном датасете разбивка по 30 мин, поэтому timeshift=48)
def shift_features_fact(df, timeshift=48):
    list_fact_columns=list(df.columns)
    list_fact_columns.remove('date_tw')
    new_df = df.copy()
    for column in list_fact_columns:
        new_df[column] = new_df[column].shift(timeshift)

    return new_df

In [5]:
# Функция для вычисления метрики mae по дням из почасовых массивов данных

def mae_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)
# Функция для вычисления метрик по дням из почасовых массивов данных

def metrics_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    mae = mean_absolute_error(y_true_grouped, y_pred_grouped)
    mape = mean_absolute_percentage_error(y_true_grouped, y_pred_grouped)
    r2 = r2_score(y_true_grouped, y_pred_grouped)
    return mae, mape, r2

def metrics_hour(y_true, y_pred): 
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mape, r2

#### 1.5 Чтение файлов с данными
Данные объединяются в один датасет

In [6]:
# читаем исходные датасеты и складываем в один
train_ds = pd.read_csv('data/train_dataset.csv')
test_ds = pd.read_csv('data/test_dataset.csv')
train_ds = pd.concat([train_ds, test_ds])

# запоминаем дату начала тестовых данных, потом также поступим и с закрытым датасетом
open_test_begin = pd.to_datetime(test_ds['date']).min()
open_test_end = pd.to_datetime(test_ds['date']).max() + pd.to_timedelta(1,'d')
print('начало открытого теста:', open_test_begin, '    конец открытого теста:', open_test_end)

начало открытого теста: 2023-04-01 00:00:00     конец открытого теста: 2023-08-01 00:00:00


#### 1.6 Формирование колонок с производными от даты

In [7]:
# преобразуем дату и делаем из нее колонки
train_ds['date'] = pd.to_datetime(train_ds['date'])
train_ds['year'] = train_ds['date'].dt.year
train_ds['month'] = train_ds['date'].dt.month
train_ds['day_of_week'] = train_ds['date'].dt.dayofweek
train_ds['day'] = train_ds['date'].dt.day
train_ds['day_of_year'] = train_ds['date'].dt.dayofyear

#### 1.7 Подгрузка данных о праздниках

In [8]:
# Добавление данных о праздниках из файла 'data/holidays.csv'

df_holidays = pd.read_csv('data/holidays.csv')
df_holidays['date'] = pd.to_datetime(df_holidays['date'])

# Assuming df_holidays and train_ds are your dataframes
train_ds = pd.merge(train_ds, df_holidays, on='date', how='left')

# Fill NaN values with 0
train_ds['holidays'].fillna(0, inplace=True)
train_ds['preholidays'].fillna(0, inplace=True)

# Convert to int
train_ds['holidays'] = train_ds['holidays'].astype(int)
train_ds['preholidays'] = train_ds['preholidays'].astype(int)

In [9]:
df_holidays['date'].max()

Timestamp('2023-12-31 00:00:00')

In [10]:
df_holidays[df_holidays.duplicated()]

,date,holidays,preholidays


#### 1.8 Формирование колонок со значением целевого признака в предыдущие дни

In [11]:
# Добавление колонок с временными лагами

# создаем столбец 'temp_last_day'
train_ds['temp_last_day'] = train_ds['temp'].shift(24)

# заполняем пропущенные значения в 'temp_last_day'
train_ds['temp_last_day'].fillna(method='bfill', inplace=True)

# создаем столбцы с временными лагами для 'target'
lags = [24, 48, 72, 7*24, 14*24]
for lag in lags:
    train_ds[f'target_lag_{lag}'] = train_ds['target'].shift(lag)

# заполняем пропущенные значения в столбцах с лагами
for lag in lags:
    train_ds[f'target_lag_{lag}'].fillna(0, inplace=True)

/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_15917/2993370237.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_ds['temp_last_day'].fillna(method='bfill', inplace=True)


#### 1.9 Формирование колонок с ВВП и данными о погоде посредством ранее описанных функций

In [12]:
# применяем функцию добавления ВВП
train_ds = add_vvp2(train_ds)

# Расшифровка прогноза в колонке 'weather_pred'
train_ds = fill_weather_columns(train_ds)


# Читаем файл с архивом фактической погоды
df_true_weather = pd.read_csv('data/preprocessing_loaded_table.csv')
display(df_true_weather)

# Форматируем колонки
df_true_weather['WW'] = df_true_weather['WW'].apply(true_weather_WW_replace)
df_true_weather['date'] = pd.to_datetime(df_true_weather['date'])
df_true_weather = df_true_weather.rename(columns={'date':'date_tw'})
# Применяем сдвиг на сутки, чтобы не заглядывать в будущее
df_true_weather = shift_features_fact(df_true_weather)
# Добавляем в датасет
train_ds['date_hours'] = train_ds.apply(row_plus_hours_to_index, axis=1)
train_ds = train_ds.merge(df_true_weather, left_on='date_hours', right_on='date_tw')
train_ds = train_ds.drop(['date_hours', 'date_tw'], axis=1)

,date,P,U,WW,Td,N,S,W,E
0,2018-12-31 00:00:00,763.5,100.0,слабый дождь,2.0,1.0,0.0,0.0,0.0
1,2018-12-31 00:30:00,764.3,93.0,слабый дождь,1.0,1.0,0.0,0.0,0.5
2,2018-12-31 01:00:00,764.3,93.0,слабый дождь,1.0,1.0,0.0,0.0,0.0
3,2018-12-31 01:30:00,765.0,93.0,слабый дождь,2.0,1.0,0.0,0.0,0.0
4,2018-12-31 02:00:00,765.0,93.0,нет осадков,2.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
82146,2023-09-30 21:30:00,763.5,82.0,нет осадков,12.0,0.0,0.0,1.0,0.0
82147,2023-09-30 22:00:00,763.5,82.0,нет осадков,12.0,0.5,0.0,1.0,0.0
82148,2023-09-30 22:30:00,763.5,77.0,сильный дождь,11.0,0.0,0.0,1.0,0.0
82149,2023-09-30 23:00:00,763.5,94.0,сильный дождь,13.0,0.5,0.0,1.0,0.0


#### 1.10 Формирование колонок со средними значениями цели и фактической температуры за предыдущий день и срезы по нему

In [13]:
#train_ds[['last_day_avg_target', 'last_day_avg_temp']] = train_ds[['date', 'target', 'temp']].groupby(by='date').transform('mean').shift(24)

#def mean_evening(values, evening=19):
#    return values[evening:].mean()

#evening_slices = [19, 22]
   
#for evening_slice in evening_slices:
#    train_ds[['last_evening_avg_target_'+str(evening_slice), 'last_evening_avg_temp_'+str(evening_slice)]] = \
#        train_ds[['date', 'target', 'temp']].groupby(by='date').transform(mean_evening, evening=evening_slice).shift(24)

In [14]:
train_ds[['last_day_avg_target', 'last_day_avg_temp']] = train_ds[['date', 'target', 'temp']].groupby(by='date').transform('mean').shift(24)

def mean_evening(values, evening=19):
    return values[evening:].mean()

evening_slices = [19, 22]
    
for evening_slice in evening_slices:
    train_ds[['last_evening_avg_target_'+str(evening_slice), 'last_evening_avg_temp_'+str(evening_slice)]] = \
        train_ds[['date', 'target', 'temp']].groupby(by='date').transform(mean_evening, evening=evening_slice).shift(24)

In [15]:
train_ds.shape

(40027, 41)

#### 1.11 Формирование колонок с почасовыми лагами для всех сформированных ранее готовых признаков

Сначала готовим список названий

In [16]:
# Отбираем признаки. Все лишние колонки здесь отбрасываем, кроме 'date', которую уберем позже 

feature_cols = list(train_ds.columns)

# выбрасываем взгляд в прошлое и расшифрованную погоду
drop_list = ['target', 'day_of_year', 'weather_pred', 'weather_fact', 'temp']

# выбрасываем признаки, найденные процедурно в процессе оптимизации
# КОМАНДЕ: здесь можно добавлять признаки на выброс с целью оптимизации
drop_list = drop_list + ['target_lag_48', 'target_lag_168'] #, 'temp_pred'] #, 'target_lag_336'] 

for name in drop_list:
    feature_cols.remove(name)

Потом добавляем лаги

In [17]:
FEATURE_WINDOW_SIZE = 24
feature_cols_no_date = feature_cols.copy()
feature_cols_no_date.remove('date')


for lag in range(1,FEATURE_WINDOW_SIZE):
    for column in feature_cols_no_date:
        train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)

/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_15917/4047160460.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_15917/4047160460.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_15917/4047160460.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

#### 1.11 Формирование колонок с лагами для цели и фактической температуры

Заполняем значениями NaN все поля этих лагов, которые относятся к текущему дню, чтобы не допустить утечки. Т.е. для каждого часа используем только лаги, которые превосходят номер часа.

In [18]:
target_lags=[1, 5, 9]

for lag in target_lags:
    train_ds['target_'+str(lag)] = train_ds.target.shift(lag).where(train_ds['time']<lag, np.NaN)
    train_ds['temp_'+str(lag)] = train_ds['temp'].shift(lag).where(train_ds['time']<lag, np.NaN)

/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_15917/1660665095.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds['target_'+str(lag)] = train_ds.target.shift(lag).where(train_ds['time']<lag, np.NaN)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_15917/1660665095.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds['temp_'+str(lag)] = train_ds['temp'].shift(lag).where(train_ds['time']<lag, np.NaN)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_15917/1660665095.py:4: Performan

#### 1.12 Демонстрация сформированного датасета

In [19]:
# Итоговый набор колонок
train_ds.columns

Index(['date', 'time', 'target', 'temp', 'temp_pred', 'weather_pred',
       'weather_fact', 'year', 'month', 'day_of_week',
       ...
       'last_evening_avg_target_19_23', 'last_evening_avg_temp_19_23',
       'last_evening_avg_target_22_23', 'last_evening_avg_temp_22_23',
       'target_1', 'temp_1', 'target_5', 'temp_5', 'target_9', 'temp_9'],
      dtype='object', length=806)

In [20]:
train_ds.head()

,date,time,target,temp,temp_pred,weather_pred,weather_fact,year,month,day_of_week,...,last_evening_avg_target_19_23,last_evening_avg_temp_19_23,last_evening_avg_target_22_23,last_evening_avg_temp_22_23,target_1,temp_1,target_5,temp_5,target_9,temp_9
0,2019-01-01,0,481.510,2.9,2.0,"пасм, ветер",ветер,2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,1,462.872,2.9,2.0,"пасм, ветер",ветер,2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01,2,449.718,2.9,2.0,"пасм, ветер",ветер,2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01,3,430.908,4.3,2.0,"пасм, ветер","ветер, пасм",2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01,4,415.163,4.3,2.0,"пасм, ветер","ветер, пасм",2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 1.13 Исключение лишних колонок

In [21]:
# Отбираем признаки. Список формируем заново из всех текущих колонок.
feature_cols = list(train_ds.columns)

# Отбрасываем колонки из ранее заготовленного списка на выброс. На этом этапе уходят колонки с утечками.
for name in drop_list:
    feature_cols.remove(name)

#### 1.14 Выделение наборов данных для обучения, валидации и тестирования

Выделялось два набора данных для обучения и валидации:
1. Обучение на данных с 2019 по 2021 с валидацией на 2022
2. Обучение на данных с 2019 по 2022 с валидацией на первом квартале 2023

Первый набор позволяет оценить влияние сезонности на обучение и предсказания, второй позволяет обучить модель на большем объеме данных и на более актуальных данных.

In [22]:
# Формируем набор датасетов для обучения и проверки

features = train_ds[feature_cols]
target = train_ds['target']

# Функция для выделения временных интервалов из таблиц признаков и целей
# на этом этапе отбрасываем колонку 'date'
def features_interval(features, target, date1, date2):
    features_interval = features[ (features['date']>=date1) & (features['date']<date2) ]
    target_interval = target[features_interval.index]
    features_interval = features_interval.drop('date', axis=1)
    return features_interval, target_interval

# для первичного подбора гиперпараметров будем обучать на 19-21 годах, валидировать по 2022
features_train, target_train = features_interval(features, target, '2019-01-01', '2022-01-01')
features_valid, target_valid = features_interval(features, target, '2022-01-01', '2023-01-01')

# отбор признаков будем производить, обучая на 19-22 и проверяя по первому кварталу 2023
# с дополнительным контролем на вариантах из первичного обучения
features_2022, target_2022 = features_interval(features, target, '2019-01-01', '2023-01-01')
features_2023, target_2023 = features_interval(features, target, '2023-01-01', open_test_begin)

# для проверки на тестовой выборке будем учиться на всем тренировочном датасете
features_all_train, target_all_train = features_interval(features, target, '2019-01-01', open_test_begin)
features_open_test, target_open_test = features_interval(features, target, open_test_begin, open_test_end)

# формируем наборы данных по кварталам 2022 года, чтобы посмотреть по ним метрику отдельно
dates = ['2022-01-01', '2022-04-01', '2022-07-01', '2022-10-01', '2023-01-01']
quarters = []
for i in range(4):
    f, t = features_interval(features, target, dates[i], dates[i+1])
    quarters.append({'features':f, 'target':t})

### 2. Обучение моделей

В настоящей работе обучаются модели LightGBM и XGBoost, финальное предсказание получается усреднением результатов.

#### 2.1 Гиперпараметры LightGBM
Были подобраны следующие значения гиперпараметров:

In [23]:
params = {'num_leaves':15, 'learning_rate':0.02, 'feature_fraction':1, 'num_iterations':NUM_ITERATIONS, 'random_state':random_state, 'objective':'regression_l1', 'n_jobs':-1}

#### 2.2 Обучение на данных за 2019-2021 годы и предсказание на 2022:

In [24]:
# Демонстрация предсказания с подобранными гиперпараметрами
# Тренируем на 19-21 годах, предсказываем за 2022

lgbm_model = lgb.LGBMRegressor(**params)
lgbm_model.fit(features_train, target_train)

y_pred = lgbm_model.predict(features_valid)
print(f'mae for days - {mae_day(target_valid, y_pred)}')
print(f'mae for hours - {mean_absolute_error(target_valid, y_pred)}')

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031829 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 74674
[LightGBM] [Info] Number of data points in the train set: 26217, number of used features: 798
[LightGBM] [Info] Start training from score 464.106506
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
mae for days - 134.5512819709116
mae for hours - 7.704952314260031


/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

In [25]:
# Предсказываем отдельно по четырем кварталам 2022 года

for i, quarter in enumerate(quarters):
    mae = mae_day(quarter['target'], lgbm_model.predict(quarter['features']))
    print(f'{i+1} квартал mae = {mae}')

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
1 квартал mae = 150.50830191521797
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1


/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

2 квартал mae = 138.5046988017369
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
3 квартал mae = 119.29967120607014
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
4 квартал mae = 137.64030313502752


/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

Как видно по предсказаниям, в задаче наблюдается значительная сезонность: предсказания на первый квартал 2022 года получились хуже, чем на прочие периоды. Какой сюрприз. В целом, из графического представления видно, что предказание в целом адекватно описывает динамику целевого признака.

In [26]:
# Предсказываем той же моделью (19-21) тренировочный кусок 2023 (первый квартал)
mae = mae_day(target_2023, lgbm_model.predict(features_2023))
print(f'mae = {mae}')

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1


mae = 131.64385081519103


/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

### 3 Проверка метрик на тестовом датасете
##### 3.1 Модель LightGBM

In [27]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете
params = {'num_leaves':15, 'learning_rate':0.02, 'feature_fraction':1, 'num_iterations':10000, 'random_state':random_state, 'objective':'regression_l1', 'n_jobs':-1}

lgbm_model_all_train = lgb.LGBMRegressor(**params)
lgbm_model_all_train.fit(features_all_train, target_all_train)

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033075 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 75442
[LightGBM] [Info] Number of data points in the train set: 37108, number of used features: 798
[LightGBM] [Info] Start training from score 473.062988


LGBMRegressor(feature_fraction=1, learning_rate=0.02, n_jobs=-1,
              num_iterations=10000, num_leaves=15, objective='regression_l1',
              random_state=12345)

In [28]:
l_predict_train = lgbm_model_all_train.predict(features_all_train)
l_predict_test = lgbm_model_all_train.predict(features_open_test)

mae_train, mape_train, r2_train = metrics_hour(target_all_train, l_predict_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, l_predict_test)

FEATURES=''
results = pd.DataFrame([[f'тренировочная LGBM {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая LGBM {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
results

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1


/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM,3.176235,0.006670,0.996797
1,тестовая LGBM,6.123915,0.014347,0.985995


In [29]:
# получение важности признаков
importance = lgbm_model_all_train.feature_importances_
feature_name = features_open_test.columns
# создание DataFrame
importance_df_lgbm = pd.DataFrame({'feature': feature_name, 'importance': importance})
# сортировка по важности
importance_df_lgbm = importance_df_lgbm.sort_values(by='importance', ascending=False)

In [30]:
importance_df_lgbm

,feature,importance
9,target_lag_24,3341
11,target_lag_336,1548
10,target_lag_72,1283
42,target_lag_24_1,950
769,target_lag_72_23,860
...,...,...
101,year_3,1
629,year_19,1
530,year_16,0
200,year_6,0


##### 3.2 Модель XGBoost

In [31]:
dop_drop_XGB = [
    #'windy_18', 'U_15', 'N_6', 'W_21', 'W_23', 'rainy_8', 'N_16', 'WW_4', 'E_7', 'S_10', 'clear_21', 'U_21', 'W_15', 'U_14', 'year_13', 
    #'E_13', 'E_5', 'windy_14', 'U_16', 'clear_15', 'W_7', 'S_22', 'W_14', 'WW_10', 'windy_21',
    #'U_10', 'S_9', 'clear_8', 'S_11', 'U_18', 'W_20', 'windy_15', 'Td_5', 'S_21', 'U_12',
    #'N_22', 'U_8', 'cloudy_16', 'U_13', 'W_18', 'S_8', 'U_11', 'N_1', 'WW_7', 'E_8', 'W_12', 'WW_5', 'W_16', 'has_rain_probability_6', 
    #'U_9', 'U_6', 'WW_14', 'E_2', 'has_rain_probability', 'W_4', 'E_11', 'W_11', 'E', 'windy_20', 'WW_13', 'WW_22', 'WW_20', 'N',
    #'WW_21', 'N_2', 'E_12', 'E_21', 'year_4', 'WW_6', 'E_14', 'E_1', 'W_2', 'W_9', 'S_12', 'WW_19', 'WW_17', 'E_16', 'S_13', 
    #'WW_15', 'year_18', 'E_23', 'E_22', 'rainy_15', 'N_10', 'WW_11', 'E_20', 'S_4', 'W_1', 'E_4', 'E_9', 'has_rain_probability_15', 
    #'WW_16', 'has_rain_probability_19', 'E_3', 'WW_12', 'WW_18', 'E_15', 'has_rain_probability_16', 'year_15', 'year_9',
    'has_rain_probability_8', 'year_8', 'has_rain_probability_11', 'year_16', 'year_7', 'has_rain_probability_3', 
    'year_17', 'has_rain_probability_13', 'year_19', 'clear_17', 'year_2', 'year_12', 'has_rain_probability_12', 'year_6']


In [32]:
drop_list_main = ['preholidays',
            #'has_rain_probability', 
             #'W', 'E'
            ]
n_values = range(1, 24)
preholidays = ['preholidays_{}'.format(n) for n in n_values]
#has_rain = ['has_rain_probability_{}'.format(n) for n in n_values]
#W_wind = ['W_{}'.format(n) for n in n_values]
#E_wind = ['E_{}'.format(n) for n in n_values]

drop_list = drop_list_main + preholidays + dop_drop_XGB #+ has_rain + W_wind + E_wind # 

feat_xgb_train = features_all_train.drop(columns=drop_list)
feat_xgb_test = features_open_test.drop(columns=drop_list)
#feat_xgb_train.columns, feat_xgb_test.columns

In [33]:
xgb_model = XGBRegressor(
    max_depth=7,
    n_estimators=1190, #n_estimators=195, #
    learning_rate=0.009, #learning_rate=0.1, #
    tree_method='exact',
    #tree_method ='gpu_hist',
    objective='reg:squarederror',
    eval_metric='rmse',
    gamma=2,
    colsample_bytree=1,
    random_state=random_state
    )

In [34]:
# Проверка метрики лучшей модели на тестовом датасете


class IterationInfoCallback(TrainingCallback):
    def __init__(self):
        self.start_time = time.time()

    def after_iteration(self, model, epoch, evals_log):
        print('Iteration:', epoch, 'Time for last iteration:', self.start_time - time.time())
        self.start_time = time.time()
        return False



xgb_model_all_train = xgb_model.fit(feat_xgb_train, target_all_train, callbacks=[IterationInfoCallback()])
xgb_predict_test = xgb_model_all_train.predict(feat_xgb_test)
xgb_predict_train = xgb_model_all_train.predict(feat_xgb_train)


/opt/anaconda3/envs/p311/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


Iteration: 0 Time for last iteration: -0.589637041091919
Iteration: 1 Time for last iteration: -0.37102198600769043
Iteration: 2 Time for last iteration: -0.3379659652709961
Iteration: 3 Time for last iteration: -0.45894694328308105
Iteration: 4 Time for last iteration: -0.32599902153015137
Iteration: 5 Time for last iteration: -0.3119640350341797
Iteration: 6 Time for last iteration: -0.3071258068084717
Iteration: 7 Time for last iteration: -0.3217008113861084
Iteration: 8 Time for last iteration: -0.31131911277770996
Iteration: 9 Time for last iteration: -0.33135199546813965
Iteration: 10 Time for last iteration: -0.32219696044921875
Iteration: 11 Time for last iteration: -0.3083341121673584
Iteration: 12 Time for last iteration: -0.30073022842407227
Iteration: 13 Time for last iteration: -0.32784414291381836
Iteration: 14 Time for last iteration: -0.33263516426086426
Iteration: 15 Time for last iteration: -0.3357889652252197
Iteration: 16 Time for last iteration: -0.3305718898773193

In [35]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, xgb_predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, xgb_predict_test )

results = pd.concat([results,
pd.DataFrame([[f'тренировочная XGB {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая XGB {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

results

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM,3.176235,0.006670,0.996797
1,тестовая LGBM,6.123915,0.014347,0.985995
0,тренировочная XGB,3.847216,0.008094,0.997486
1,тестовая XGB,6.143893,0.014411,0.984871


In [36]:
drop_list_cat = ['year_5', 'temp_1', 'clear_16', 'has_rain_probability_19', 'year_19', 'year_20', 'N_14', 'has_rain_probability_12', 'E_4', 'clear_15', 'windy_20', 'year_12', 'has_rain_probability_17', 'year_16', 'year_4', 'rainy_16', 'has_rain_probability_11', 'clear_20', 'clear_14', 'has_rain_probability_8', 'E_9', 'E_3', 'E_15', 'windy_21', 'N_12', 'year_3', 'N_18', 'has_rain_probability_18', 'clear_7', 'WW_15', 'has_rain_probability_21', 'N_10', 'holidays_15', 'E_21', 'S_15', 'N_13', 'windy_14', 'E_10', 'S_9', 'E_5', 'WW_12', 'N_19', 'WW_14', 'WW_21', 'S_8', 'N_7', 'windy_19', 'has_rain_probability_15', 'S_14', 'E_18', 'WW_23', 'WW_13', 'year_6', 'S_12', 'clear_18', 'has_rain_probability_14', 'W_8', 'S_21', 'WW_20', 'windy_13', 'clear_8', 'rainy_17', 'WW_18', 'N_8', 'has_rain_probability_10', 'S_13', 'year_13', 'rainy_18', 'W_21', 'rainy_15', 'E_19', 'N_11', 'N_9', 'year_21', 'W_6', 'E_8', 'S_22', 'S_17', 'WW_22', 'clear_6', 'S_3', 'year_17', 'WW_17', 'clear_19', 'W_15', 'W_11', 'WW_16', 'year_22', 'WW_8', 'WW_6', 'year_18', 'has_rain_probability_20', 'WW_9', 'rainy_11', 'year_7', 'rainy_12', 'cloudy_20', 'E_2', 'W_12', 'W_13', 'W_4']
#drop_list_cat_2 = ['has_rain_probability_5', 'has_rain_probability', 'has_rain_probability_1', 'E_12', 'has_rain_probability_6', 'has_rain_probability_13', 'year_11', 'clear_4', 'N_5', 'windy_15', 'clear_21', 'has_rain_probability_9', 'N_17', 'WW_7', 'W_9', 'clear_12', 'cloudy_16', 'windy_5', 'E_22', 'E_16', 'W_10', 'E', 'N_6', 'N_2', 'E_6', 'N_3', 'clear_17', 'has_rain_probability_22', 'S_5', 'rainy_14', 'clear_13', 'N_4', 'temp_5', 'W_22', 'target_5', 'has_rain_probability_7', 'windy_8', 'S_11', 'N_16', 'W', 'W_19', 'E_11', 'S_10', 'year_8', 'windy_22', 'clear_3', 'E_20', 'W_1', 'cloudy_22', 'clear_22', 'cloudy_21', 'E_23', 'Td_3', 'cloudy_18', 'has_rain_probability_16', 'windy_18', 'WW_10', 'Td_4', 'W_16', 'E_14', 'W_23', 'windy_17', 'WW_19', 'WW_11', 'year_15', 'W_7', 'rainy_8', 'S_7', 'windy_16']
drop_list_main = ['preholidays']

In [37]:
drop_list_cat = drop_list_cat + drop_list_main + preholidays #+ drop_list_cat_2

drop_list_cat

['year_5',
 'temp_1',
 'clear_16',
 'has_rain_probability_19',
 'year_19',
 'year_20',
 'N_14',
 'has_rain_probability_12',
 'E_4',
 'clear_15',
 'windy_20',
 'year_12',
 'has_rain_probability_17',
 'year_16',
 'year_4',
 'rainy_16',
 'has_rain_probability_11',
 'clear_20',
 'clear_14',
 'has_rain_probability_8',
 'E_9',
 'E_3',
 'E_15',
 'windy_21',
 'N_12',
 'year_3',
 'N_18',
 'has_rain_probability_18',
 'clear_7',
 'WW_15',
 'has_rain_probability_21',
 'N_10',
 'holidays_15',
 'E_21',
 'S_15',
 'N_13',
 'windy_14',
 'E_10',
 'S_9',
 'E_5',
 'WW_12',
 'N_19',
 'WW_14',
 'WW_21',
 'S_8',
 'N_7',
 'windy_19',
 'has_rain_probability_15',
 'S_14',
 'E_18',
 'WW_23',
 'WW_13',
 'year_6',
 'S_12',
 'clear_18',
 'has_rain_probability_14',
 'W_8',
 'S_21',
 'WW_20',
 'windy_13',
 'clear_8',
 'rainy_17',
 'WW_18',
 'N_8',
 'has_rain_probability_10',
 'S_13',
 'year_13',
 'rainy_18',
 'W_21',
 'rainy_15',
 'E_19',
 'N_11',
 'N_9',
 'year_21',
 'W_6',
 'E_8',
 'S_22',
 'S_17',
 'WW_22',
 'clea

In [38]:
feat_cat_test =  features_open_test.drop(columns=drop_list_cat)
feat_cat_train = features_all_train.drop(columns=drop_list_cat)

In [39]:
catboost_model = CatBoostRegressor(
    iterations=23000,
    learning_rate=0.015,
    depth=6,
    subsample=0.65,
    colsample_bylevel=0.95,
    min_data_in_leaf=51,
    l2_leaf_reg=9
)

catboost_model.fit(feat_cat_train , target_all_train)
"""
params_catboost = {
    'iterations': [100, 200, 300, 500, 1000, 3000, 10000],
    'learning_rate': [0.001, 0.005, 0.01, 0.015, 0.02, 0.03, 0.05, 0.1],
    'depth': range(1, 10, 1),
    'subsample': [x / 100.0 for x in range(5, 100, 10)],
    'colsample_bylevel': [x / 100.0 for x in range(5, 100, 10)],
    'min_data_in_leaf': range(1, 100, 10),
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    #'min_child_samples': [1, 4, 8, 16, 32]
}

grid_search_catboost = RandomizedSearchCV(
    catboost_model,
    cv = TimeSeriesSplit(n_splits=4),
    param_distributions=params_catboost,
    n_jobs=-1,
    random_state=random_state,
    scoring='neg_mean_absolute_error'
)

grid_search_catboost.fit(features_train, target_train)"""

0:	learn: 100.1330132	total: 82.2ms	remaining: 31m 31s
1:	learn: 98.8216133	total: 107ms	remaining: 20m 32s
2:	learn: 97.5124716	total: 131ms	remaining: 16m 47s
3:	learn: 96.2239934	total: 155ms	remaining: 14m 48s
4:	learn: 94.9620050	total: 180ms	remaining: 13m 47s
5:	learn: 93.7293400	total: 204ms	remaining: 13m
6:	learn: 92.4949322	total: 228ms	remaining: 12m 28s
7:	learn: 91.2929641	total: 253ms	remaining: 12m 8s
8:	learn: 90.0725597	total: 282ms	remaining: 11m 59s
9:	learn: 88.8928864	total: 305ms	remaining: 11m 40s
10:	learn: 87.7333891	total: 328ms	remaining: 11m 26s
11:	learn: 86.6028491	total: 356ms	remaining: 11m 21s
12:	learn: 85.4649452	total: 379ms	remaining: 11m 9s
13:	learn: 84.3530342	total: 402ms	remaining: 11m
14:	learn: 83.2683863	total: 427ms	remaining: 10m 54s
15:	learn: 82.1820219	total: 453ms	remaining: 10m 51s
16:	learn: 81.1426084	total: 476ms	remaining: 10m 42s
17:	learn: 80.0851611	total: 498ms	remaining: 10m 35s
18:	learn: 79.0395376	total: 521ms	remaining: 

"\nparams_catboost = {\n    'iterations': [100, 200, 300, 500, 1000, 3000, 10000],\n    'learning_rate': [0.001, 0.005, 0.01, 0.015, 0.02, 0.03, 0.05, 0.1],\n    'depth': range(1, 10, 1),\n    'subsample': [x / 100.0 for x in range(5, 100, 10)],\n    'colsample_bylevel': [x / 100.0 for x in range(5, 100, 10)],\n    'min_data_in_leaf': range(1, 100, 10),\n    'l2_leaf_reg': [1, 3, 5, 7, 9],\n    #'min_child_samples': [1, 4, 8, 16, 32]\n}\n\ngrid_search_catboost = RandomizedSearchCV(\n    catboost_model,\n    cv = TimeSeriesSplit(n_splits=4),\n    param_distributions=params_catboost,\n    n_jobs=-1,\n    random_state=random_state,\n    scoring='neg_mean_absolute_error'\n)\n\ngrid_search_catboost.fit(features_train, target_train)"

In [40]:
cat_predict_test = catboost_model.predict(feat_cat_test)
cat_predict_train = catboost_model.predict(feat_cat_train)

In [41]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, cat_predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, cat_predict_test )

results = pd.concat([results,
pd.DataFrame([[f'тренировочная CAT {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая CAT {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

In [42]:
results

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM,3.176235,0.006670,0.996797
1,тестовая LGBM,6.123915,0.014347,0.985995
0,тренировочная XGB,3.847216,0.008094,0.997486
1,тестовая XGB,6.143893,0.014411,0.984871
0,тренировочная CAT,1.653186,0.003525,0.999549
1,тестовая CAT,5.949680,0.014065,0.987062


In [43]:
feature_importances_cat = catboost_model.get_feature_importance()

# Создаем DataFrame
importances_df_cat = pd.DataFrame({
    'feature': feat_cat_train.columns,
    'importance': feature_importances_cat
})

# Сортируем DataFrame по важности признаков
importances_df_cat = importances_df_cat.sort_values(by='importance', ascending=True)

In [44]:
importances_df_cat

,feature,importance
169,has_rain_probability_5,0.000086
17,has_rain_probability,0.000173
49,has_rain_probability_1,0.000229
368,E_12,0.000343
198,has_rain_probability_6,0.000418
...,...,...
672,temp_9,1.354414
10,target_lag_336,3.576399
40,target_lag_24_1,4.562388
9,target_lag_72,8.517591


In [45]:
# Фильтруем DataFrame
low_importance_features = importances_df_cat.loc[importances_df_cat['importance'] < 0.002, 'feature']

# Выводим список признаков
print(low_importance_features.tolist())
print(len(low_importance_features))

['has_rain_probability_5', 'has_rain_probability', 'has_rain_probability_1', 'E_12', 'has_rain_probability_6', 'year_11', 'has_rain_probability_13', 'clear_4', 'windy_15', 'N_5', 'clear_21', 'has_rain_probability_9', 'N_17', 'clear_12', 'WW_7', 'W_9', 'windy_5', 'cloudy_16', 'E_22', 'E_16', 'W_10', 'E', 'N_2', 'N_6', 'E_6', 'has_rain_probability_22', 'clear_17', 'clear_13', 'rainy_14', 'N_3', 'N_4', 'S_5', 'has_rain_probability_7', 'target_5', 'W_22', 'temp_5', 'windy_8', 'N_16', 'W_19', 'year_8', 'W', 'S_10', 'windy_22', 'clear_3', 'E_11', 'E_20', 'S_11', 'W_1', 'cloudy_22', 'clear_22', 'cloudy_21', 'E_23', 'has_rain_probability_16', 'Td_3', 'cloudy_18', 'windy_17', 'WW_10', 'Td_4', 'WW_11', 'windy_18', 'E_14', 'W_23', 'year_15', 'WW_19', 'W_16', 'rainy_8']
66


In [46]:
importance = xgb_model.feature_importances_
# предположим, что 'X' - это ваши данные
feature_name = feat_xgb_test.columns
# создание DataFrame
importance_df_xgb = pd.DataFrame({'feature': feature_name, 'importance': importance})
# сортировка по важности
importance_df_xgb = importance_df_xgb.sort_values(by='importance', ascending=False)


In [47]:
# Фильтруем DataFrame
low_importance_xgb = importance_df_xgb.loc[importance_df_xgb['importance'] < 0.0001, 'feature']

# Выводим список признаков
print(low_importance_xgb.tolist())
print(len(low_importance_xgb))

['windy_18', 'U_15', 'N_6', 'W_21', 'W_23', 'rainy_8', 'N_16', 'WW_4', 'E_7', 'S_10', 'clear_21', 'U_21', 'W_15', 'U_14', 'year_13', 'E_13', 'E_5', 'windy_14', 'U_16', 'clear_15', 'W_7', 'S_22', 'W_14', 'WW_10', 'windy_21', 'U_10', 'S_9', 'clear_8', 'S_11', 'U_18', 'W_20', 'windy_15', 'Td_5', 'S_21', 'U_12', 'N_22', 'U_8', 'cloudy_16', 'U_13', 'W_18', 'S_8', 'U_11', 'N_1', 'WW_7', 'E_8', 'W_12', 'WW_5', 'W_16', 'has_rain_probability_6', 'U_9', 'U_6', 'WW_14', 'E_2', 'has_rain_probability', 'W_4', 'E_11', 'W_11', 'E', 'windy_20', 'WW_13', 'WW_22', 'WW_20', 'N', 'WW_21', 'N_2', 'E_12', 'E_21', 'year_4', 'WW_6', 'E_14', 'E_1', 'W_2', 'W_9', 'S_12', 'WW_19', 'WW_17', 'E_16', 'S_13', 'WW_15', 'year_18', 'E_23', 'E_22', 'rainy_15', 'N_10', 'WW_11', 'E_20', 'S_4', 'W_1', 'E_4', 'E_9', 'has_rain_probability_15', 'WW_16', 'has_rain_probability_19', 'E_3', 'WW_12', 'WW_18', 'E_15', 'has_rain_probability_16', 'year_15', 'year_9']
100


In [48]:
importance_df_xgb.tail(50)

,feature,importance
208,U_6,0.000075
458,WW_14,0.000075
88,E_2,0.000075
17,has_rain_probability,0.000074
150,W_4,0.000073
370,E_11,0.000072
369,W_11,0.000071
25,E,0.000071
640,windy_20,0.000069
426,WW_13,0.000069


In [49]:
merged_df = pd.merge(importance_df_lgbm, importance_df_xgb, on='feature', how='outer', suffixes=('_lgbm', '_xgb'))

In [50]:
merged_df

,feature,importance_lgbm,importance_xgb
0,target_lag_24,3341,0.503236
1,target_lag_336,1548,0.006494
2,target_lag_72,1283,0.002909
3,target_lag_24_1,950,0.000855
4,target_lag_72_23,860,0.002217
...,...,...,...
793,year_3,1,0.001493
794,year_19,1,NaN
795,year_16,0,NaN
796,year_6,0,NaN


In [51]:
# Замена NaN на 0 в столбце 'importance_xgb'
merged_df['importance_xgb'].fillna(0, inplace=True)

# Нормализация важности признаков
merged_df['importance_lgbm'] = merged_df['importance_lgbm'] / merged_df['importance_lgbm'].max()
merged_df['importance_xgb'] = merged_df['importance_xgb'] / merged_df['importance_xgb'].max()

# Создание столбца 'importance_ensemble', который является средним значением 'importance_lgbm' и 'importance_xgb'
merged_df['importance_ensemble'] = (merged_df['importance_lgbm'] + merged_df['importance_xgb']) / 2

In [52]:
merged_df

,feature,importance_lgbm,importance_xgb,importance_ensemble
0,target_lag_24,1.000000,1.000000,1.000000
1,target_lag_336,0.463334,0.012905,0.238120
2,target_lag_72,0.384017,0.005781,0.194899
3,target_lag_24_1,0.284346,0.001698,0.143022
4,target_lag_72_23,0.257408,0.004405,0.130906
...,...,...,...,...
793,year_3,0.000299,0.002966,0.001633
794,year_19,0.000299,0.000000,0.000150
795,year_16,0.000000,0.000000,0.000000
796,year_6,0.000000,0.000000,0.000000


In [53]:
!pip install openpyxl

In [54]:
merged_df.to_excel('feature_importance.xlsx')

In [55]:
merged_df.to_csv('feature_importance.csv')

### 4 Объединяем результаты ансамбля моделей

In [56]:
predict_simple_ensemble_train = (xgb_predict_train + l_predict_train)/2
predict_simple_ensemble_test = (xgb_predict_test + l_predict_test)/2

In [57]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, predict_simple_ensemble_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, predict_simple_ensemble_test)

results_ensemble = results
results_ensemble = pd.concat([results_ensemble,
pd.DataFrame([[f'тренировочная ансамбля LGBM и XGB  {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая ансамбля LGBM и XGB {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

In [58]:
display(results_ensemble)

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM,3.176235,0.006670,0.996797
1,тестовая LGBM,6.123915,0.014347,0.985995
0,тренировочная XGB,3.847216,0.008094,0.997486
1,тестовая XGB,6.143893,0.014411,0.984871
0,тренировочная CAT,1.653186,0.003525,0.999549
1,тестовая CAT,5.949680,0.014065,0.987062
0,тренировочная ансамбля LGBM и XGB,3.398171,0.007138,0.997498
1,тестовая ансамбля LGBM и XGB,5.962481,0.013958,0.986196


In [59]:
predict_simple_ensemble_train = (cat_predict_train + l_predict_train)/2
predict_simple_ensemble_test = (cat_predict_test + l_predict_test)/2

In [60]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, predict_simple_ensemble_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, predict_simple_ensemble_test)


results_ensemble = pd.concat([results_ensemble,
pd.DataFrame([[f'тренировочная ансамбля CatBoost и LGBM  {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая ансамбля CatBoost и LGBM {FEATURES} ', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

In [61]:
display(results_ensemble)

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM,3.176235,0.006670,0.996797
1,тестовая LGBM,6.123915,0.014347,0.985995
0,тренировочная XGB,3.847216,0.008094,0.997486
1,тестовая XGB,6.143893,0.014411,0.984871
0,тренировочная CAT,1.653186,0.003525,0.999549
1,тестовая CAT,5.949680,0.014065,0.987062
0,тренировочная ансамбля LGBM и XGB,3.398171,0.007138,0.997498
1,тестовая ансамбля LGBM и XGB,5.962481,0.013958,0.986196
0,тренировочная ансамбля CatBoost и LGBM,2.308165,0.004868,0.998719
1,тестовая ансамбля CatBoost и LGBM,5.873769,0.013813,0.987240


In [62]:
predict_simple_ensemble_train = xgb_predict_train*0.3 + l_predict_train*0.23 + cat_predict_train*0.47
predict_simple_ensemble_test = xgb_predict_test*0.3 + l_predict_test*0.23 + cat_predict_test*0.47

In [63]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, predict_simple_ensemble_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, predict_simple_ensemble_test)


results_ensemble = pd.concat([results_ensemble,
pd.DataFrame([[f'тренировочная ансамбля CatBoost, LGBM и XGBoost {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая ансамбля CatBoost, LGBM и XGBoost {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

In [64]:
display(results_ensemble)

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM,3.176235,0.006670,0.996797
1,тестовая LGBM,6.123915,0.014347,0.985995
0,тренировочная XGB,3.847216,0.008094,0.997486
1,тестовая XGB,6.143893,0.014411,0.984871
0,тренировочная CAT,1.653186,0.003525,0.999549
1,тестовая CAT,5.949680,0.014065,0.987062
0,тренировочная ансамбля LGBM и XGB,3.398171,0.007138,0.997498
1,тестовая ансамбля LGBM и XGB,5.962481,0.013958,0.986196
0,тренировочная ансамбля CatBoost и LGBM,2.308165,0.004868,0.998719
1,тестовая ансамбля CatBoost и LGBM,5.873769,0.013813,0.987240
